# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Paso 1: Leer el dataset
file_path = "tic-tac-toe.csv"  # Asegúrate de tener el archivo en el directorio
data = pd.read_csv(file_path)

# Inspección inicial del dataset
print("Primeras filas del dataset:")
print(data.head())
print("\nInformación del dataset:")
print(data.info())

# Paso 2: Convertir valores categóricos a numéricos
# Usamos LabelEncoder para convertir las columnas
label_encoder = LabelEncoder()

for col in data.columns[:-1]:  # Excluye la columna de clase
    data[col] = label_encoder.fit_transform(data[col])

# Convertir la columna 'class' a binaria (1 si Player X gana, 0 si no)
data['class'] = data['class'].apply(lambda x: 1 if x == 'positive' else 0)

# Verificar los datos después de la conversión
print("\nDatos después de convertir valores categóricos:")
print(data.head())

# Paso 3: Separar las entradas (X) y salidas (y)
X = data.iloc[:, :-1].values  # Todas las columnas menos la última
y = data.iloc[:, -1].values  # Solo la columna 'class'

# Paso 4: Normalizar los datos de entrada
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Paso 5: Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Confirmar formas de los conjuntos de datos
print("\nFormas de los datos:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


Primeras filas del dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None

Datos después de convertir valores categóricos:
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   2   2   2   2   1   1   2   1   1      0

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Paso 2.1: Asegúrate de que los datos ya están divididos en X_train, X_test, y_train, y_test
# Esto se realizó en el paso anterior.

# Paso 2.2: Crear un modelo secuencial
model = Sequential()

# Paso 2.3: Agregar capas al modelo
# Capa de entrada con 9 entradas (para las 9 casillas de tic-tac-toe)
model.add(Dense(64, input_dim=9, activation='relu'))  # Primera capa oculta
model.add(Dense(32, activation='relu'))              # Segunda capa oculta
model.add(Dense(1, activation='sigmoid'))            # Capa de salida (para clasificación binaria)

# Paso 2.4: Compilar el modelo
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Paso 2.5: Entrenar el modelo
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=8, 
                    validation_split=0.2, 
                    verbose=1)

# Paso 2.6: Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nPérdida en el conjunto de prueba: {loss:.4f}")
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# Paso 2.7: Guardar el modelo
model.save("tic-tac-toe.keras")
print("\nModelo guardado como 'tic-tac-toe.keras'.")


Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5885 - loss: 0.6299 - val_accuracy: 1.0000 - val_loss: 0.0728
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0450 - val_accuracy: 1.0000 - val_loss: 0.0161
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0117 - val_accuracy: 1.0000 - val_loss: 0.0062
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 6.8056e-04
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 5.2457e-04 - val_accuracy: 1.0000 - val_loss: 4.1388e-04
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 3.3242e-04 - val_accuracy: 1.

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [5]:
import numpy as np
from tensorflow.keras.models import load_model

# Paso 3.1: Cargar el modelo guardado
# Asegúrate de usar la extensión correcta según cómo guardaste el modelo
model = load_model("tic-tac-toe.keras")  # O "tic-tac-toe.h5" si usaste esa extensión

# Paso 3.2: Seleccionar filas aleatorias del conjunto de prueba
random_indices = np.random.choice(len(X_test), size=5, replace=False)
random_samples = X_test[random_indices]
random_labels = y_test[random_indices]

# Paso 3.3: Hacer predicciones en las filas seleccionadas
predictions = model.predict(random_samples)
predictions = np.round(predictions).astype(int)  # Redondear para obtener 0 o 1

# Paso 3.4: Mostrar resultados
for i, (sample, prediction, actual) in enumerate(zip(random_samples, predictions, random_labels)):
    print(f"Muestra {i+1}:")
    print(f"  Entrada: {sample}")
    print(f"  Predicción: {prediction[0]} (1 = gana X, 0 = no gana X)")
    print(f"  Etiqueta real: {actual}")
    print()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Muestra 1:
  Entrada: [ 1.00322257 -1.41958965 -0.28682739 -1.41958965  0.93037142 -0.16731812
  1.00322257 -1.41958965 -0.28682739]
  Predicción: 0 (1 = gana X, 0 = no gana X)
  Etiqueta real: 0

Muestra 2:
  Entrada: [ 1.00322257 -1.41958965 -0.28682739  1.08495342  0.93037142 -0.16731812
 -1.57687736 -0.16731812  1.00322257]
  Predicción: 0 (1 = gana X, 0 = no gana X)
  Etiqueta real: 0

Muestra 3:
  Entrada: [-1.57687736 -0.16731812 -0.28682739  1.08495342  0.93037142  1.08495342
 -1.57687736 -0.16731812  1.00322257]
  Predicción: 0 (1 = gana X, 0 = no gana X)
  Etiqueta real: 0

Muestra 4:
  Entrada: [ 1.00322257  1.08495342 -0.28682739 -0.16731812  0.93037142 -0.16731812
  1.00322257 -0.16731812  1.00322257]
  Predicción: 0 (1 = gana X, 0 = no gana X)
  Etiqueta real: 0

Muestra 5:
  Entrada: [ 1.00322257 -0.16731812 -1.57687736 -0.16731812  0.93037142 -0.16731812
 -1.57687736  1.08495342  1.00322257]
  Predicción: 0 (1 = gana X, 0 = no gana 

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Paso 1: Crear un optimizador personalizado con una tasa de aprendizaje ajustada
learning_rate = 0.001  # Puedes experimentar con valores como 0.0001 o 0.01
optimizer = Adam(learning_rate=learning_rate)

# Paso 2: Crear el modelo mejorado
model = Sequential()

# Agregar capas adicionales y ajustar el número de neuronas
model.add(Dense(128, input_dim=9, activation='relu'))  # Capa oculta 1
model.add(Dropout(0.3))  # Regularización para prevenir sobreajuste
model.add(Dense(64, activation='relu'))  # Capa oculta 2
model.add(Dense(32, activation='relu'))  # Capa oculta 3
model.add(Dense(1, activation='sigmoid'))  # Capa de salida

# Paso 3: Compilar el modelo con el optimizador ajustado
model.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Paso 4: Entrenar el modelo con más épocas
history = model.fit(X_train, y_train, 
                    epochs=100,  # Incrementar el número de épocas
                    batch_size=16,  # Puedes ajustar el tamaño del batch
                    validation_split=0.2, 
                    verbose=1)

# Paso 5: Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nPérdida en el conjunto de prueba: {loss:.4f}")
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# Paso 6: Guardar el modelo mejorado
model.save("tic-tac-toe-improved.keras")
print("\nModelo mejorado guardado como 'tic-tac-toe-improved.keras'.")


Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8489 - loss: 0.4464 - val_accuracy: 1.0000 - val_loss: 0.0137
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 6.7099e-04
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 8.6131e-04 - val_accuracy: 1.0000 - val_loss: 3.9655e-04
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 5.3994e-04 - val_accuracy: 1.0000 - val_loss: 2.5844e-04
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 3.3710e-04 - val_accuracy: 1.0000 - val_loss: 1.7719e-04
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.4424e-04 - val_accuracy: 1.0000 - val_loss: 1.2872e-04
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - los

**Which approach(es) did you find helpful to improve your model performance?**

In [7]:
from tensorflow.keras.optimizers import Adam

# Crear un modelo mejorado
model_improved = Sequential()

# Agregar más capas al modelo
model_improved.add(Dense(128, input_dim=9, activation='relu'))  # Primera capa oculta
model_improved.add(Dense(64, activation='relu'))               # Segunda capa oculta
model_improved.add(Dense(32, activation='relu'))               # Tercera capa oculta
model_improved.add(Dense(1, activation='sigmoid'))             # Capa de salida

# Ajustar el optimizador con una tasa de aprendizaje personalizada
optimizer = Adam(learning_rate=0.001)

# Compilar el modelo mejorado
model_improved.compile(optimizer=optimizer, 
                       loss='binary_crossentropy', 
                       metrics=['accuracy'])

# Entrenar el modelo con más épocas
history_improved = model_improved.fit(X_train, y_train, 
                                      epochs=100, 
                                      batch_size=8, 
                                      validation_split=0.2, 
                                      verbose=1)

# Evaluar el modelo mejorado
loss_improved, accuracy_improved = model_improved.evaluate(X_test, y_test, verbose=0)
print(f"\nPérdida en el conjunto de prueba (modelo mejorado): {loss_improved:.4f}")
print(f"Precisión en el conjunto de prueba (modelo mejorado): {accuracy_improved:.4f}")


Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8588 - loss: 0.3119 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 6.0717e-04
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 5.1976e-04 - val_accuracy: 1.0000 - val_loss: 3.0949e-04
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.5798e-04 - val_accuracy: 1.0000 - val_loss: 1.8292e-04
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.5817e-04 - val_accuracy: 1.0000 - val_loss: 1.1910e-04
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.0303e-04 - val_accuracy: 1.0000 - val_loss: 8.1416e-05
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 7.3959e-05 - val_accuracy: 1.0000 - val_loss: 5.8598e-05
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.00